### Memory

In some applications like chatbots, it is important to remember previous interactions to keep the whole context of a conversation. Memory does provide you an easy way to handle this

In [19]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [20]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

history.add_user_message("hi!")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello my friend!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [21]:
# Modern approach: Use InMemoryChatMessageHistory directly as memory
# This replaces the legacy ConversationBufferMemory
memory = InMemoryChatMessageHistory()
memory.add_user_message("hi!")
memory.add_ai_message("hello my friend!")

# Access messages
memory.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello my friend!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Create a prompt template that includes message history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the base chain
chain = prompt | llm | StrOutputParser()

# Create a session history store
session_history = {}

def get_session_history(session_id: str):
    if session_id not in session_history:
        session_history[session_id] = InMemoryChatMessageHistory()
    return session_history[session_id]

# Wrap the chain with message history
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Invoke with a session ID
conversation.invoke(
    {"input": "What is the capital of France?"},
    config={"configurable": {"session_id": "session1"}}
)

'The capital of France is **Paris**.'

In [24]:
# Continue the conversation with the same session ID
conversation.invoke(
    {"input": "Whats the best food there?"},
    config={"configurable": {"session_id": "session1"}}
)

'That\'s a fantastic question, but "best" is incredibly subjective when it comes to French food – it\'s all so delicious! However, I can tell you about some of the most iconic, beloved, and must-try foods you\'ll find in Paris and throughout France:\n\n**Savory Delights:**\n\n1.  **Steak Frites:** A classic bistro staple. A perfectly cooked steak (often bavette, entrecôte, or faux-filet) served with a generous pile of crispy, golden French fries. Simple, yet incredibly satisfying.\n2.  **Coq au Vin:** Chicken braised with red wine (often Burgundy), mushrooms, lardons (bacon), and garlic. A rich, hearty, and deeply flavorful dish.\n3.  **Boeuf Bourguignon:** Similar to Coq au Vin, but with beef (usually beef chuck) braised in red wine, often with carrots, onions, and mushrooms. Another incredibly comforting and traditional stew.\n4.  **French Onion Soup (Soupe à l\'oignon gratinée):** A rich beef broth base with caramelized onions, topped with a crouton and a thick layer of melted Gruyè

### Conversation Summary Memory

When inputs get long, we might not want to send the whole conversation, but rather a summary. In modern LangChain, we implement this using a custom approach with InMemoryChatMessageHistory and manual summarization.

In [25]:
# Modern approach: Implement summary memory manually using LCEL
review = """I ordered Pizza Salami for 9.99$ and it was awesome! 
The pizza was delivered on time and was still hot when I received it. 
The crust was thin and crispy, and the toppings were fresh and flavorful. 
The Salami was well-cooked and complemented the cheese perfectly. 
The price was reasonable and I believe I got my money's worth. 
Overall, I am very satisfied with my order and I would recommend this pizza place to others."""

# Create a summary memory using InMemoryChatMessageHistory
summary_memory = InMemoryChatMessageHistory()
summary_memory.add_user_message("Hello, how can I help you today?")
summary_memory.add_ai_message("Could you analyze a review for me?")
summary_memory.add_user_message("Sure, I'd be happy to. Could you provide the review?")
summary_memory.add_ai_message(review)

# View the messages
summary_memory.messages

[HumanMessage(content='Hello, how can I help you today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Could you analyze a review for me?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content="Sure, I'd be happy to. Could you provide the review?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I ordered Pizza Salami for 9.99$ and it was awesome! \nThe pizza was delivered on time and was still hot when I received it. \nThe crust was thin and crispy, and the toppings were fresh and flavorful. \nThe Salami was well-cooked and complemented the cheese perfectly. \nThe price was reasonable and I believe I got my money's worth. \nOverall, I am very satisfied with my order and I would recommend this pizza place to others.", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [26]:
# Create a conversation chain with summary using LCEL approach
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | llm | StrOutputParser()

# For summary memory, we use the same session history approach
def get_summary_session_history(session_id: str):
    if session_id not in session_history:
        session_history[session_id] = summary_memory
    return session_history[session_id]

conversation_with_summary = RunnableWithMessageHistory(
    chain,
    get_summary_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

conversation_with_summary.invoke(
    {"input": "Thank you very much!"},
    config={"configurable": {"session_id": "summary_session"}}
)

'You\'re welcome! Here\'s an analysis of the review you provided:\n\n---\n\n**Review Analysis: Pizza Salami ($9.99)**\n\n**Overall Sentiment:**\nThis is an **overwhelmingly positive** review, indicating high customer satisfaction. The reviewer explicitly states "it was awesome," "very satisfied," and "would recommend this pizza place to others."\n\n**Key Positive Points:**\n\n1.  **Food Quality (Pizza Salami):**\n    *   **Taste:** Described as "awesome" and "flavorful."\n    *   **Crust:** Praised for being "thin and crispy."\n    *   **Toppings:** Noted as "fresh and flavorful."\n    *   **Salami:** Specifically mentioned as "well-cooked" and "complemented the cheese perfectly."\n\n2.  **Delivery Experience:**\n    *   **Punctuality:** "Delivered on time."\n    *   **Temperature:** "Still hot when I received it."\n\n3.  **Value for Money:**\n    *   **Price:** Considered "reasonable" for $9.99.\n    *   **Perception:** The reviewer felt they "got my money\'s worth."\n\n**Specific Pro

In [27]:
# View the memory messages
summary_memory.messages

[HumanMessage(content='Hello, how can I help you today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Could you analyze a review for me?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content="Sure, I'd be happy to. Could you provide the review?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I ordered Pizza Salami for 9.99$ and it was awesome! \nThe pizza was delivered on time and was still hot when I received it. \nThe crust was thin and crispy, and the toppings were fresh and flavorful. \nThe Salami was well-cooked and complemented the cheese perfectly. \nThe price was reasonable and I believe I got my money's worth. \nOverall, I am very satisfied with my order and I would recommend this pizza place to others.", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Thank you very much!', additional_kwargs={}, response_metadata={}),
 AIMessag